In [3]:
import os
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from collections import defaultdict

from models.Resnet_LSTM import ResNetLSTM
from datasets.data_loader import WikiArtDataset, collate_skip_none
from datasets.csv_data_loader import csv_Dataset

In [1]:
def analyze_model_uncertainty(task="artist", file="Artist", top_n=20):
    # === CONFIG ===
    image_root = os.path.join("datasets", "wikiart")
    class_file = os.path.join("datasets", file, f"{task}_class")

    if task == "artist":
        train_file = os.path.join("datasets", file, f"{task}_train")
        val_file = os.path.join("datasets", file, f"{task}_val")
    elif task in ["genre", "style"]:
        train_file = os.path.join("datasets", file, f"{task}_train.csv")
        val_file = os.path.join("datasets", file, f"{task}_val.csv")
    else:
        raise ValueError(f"Unsupported task: {task}")

    model_path = os.path.join("checkpoints", f"{task}_best_model.pt")
    outlier_dir = "outliers"
    os.makedirs(outlier_dir, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    class_names = [line.strip() for line in open(class_file, encoding='utf-8')]
    num_classes = len(class_names)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    # === Load data and model ===
    if task == "artist":
        dataset = WikiArtDataset(val_file, image_root, transform)
    elif task in ["genre", "style"]:
        dataset = csv_Dataset(val_file, image_root, transform)

    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_skip_none)

    model = ResNetLSTM(num_classes=num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # === Analyze uncertainty ===
    entropy_list = []
    softmax = torch.nn.Softmax(dim=1)
    error_count = defaultdict(int)
    entropy_count = defaultdict(int)
    total_count = defaultdict(int)

    with torch.no_grad():
        for idx, batch in enumerate(tqdm(dataloader, desc="Analyzing")):
            if batch is None:
                continue
            image, label = batch
            image, label = image.to(device), label.to(device)
            output = model(image)
            prob = softmax(output)

            entropy = -torch.sum(prob * torch.log(prob + 1e-9), dim=1).item()
            predicted = prob.argmax(dim=1).item()
            true_label = label.item()

            entropy_list.append((entropy, predicted, true_label, dataset.samples[idx][0]))

            true_class = class_names[true_label]
            total_count[true_class] += 1
            if predicted != true_label:
                error_count[true_class] += 1
            if entropy > 2.0:  # ⬅️ 可以視情況調整 entropy 門檻
                entropy_count[true_class] += 1

    # === Sort by highest entropy (most uncertain) ===
    entropy_list.sort(reverse=True, key=lambda x: x[0])

    # === Save top-N outliers ===
    print(f"\nTop {top_n} most uncertain predictions (possible outliers):")
    for i in range(min(top_n, len(entropy_list))):
        ent, pred, true, path = entropy_list[i]
        print(f"[{i+1}] Entropy: {ent:.4f} | True: {true} | Pred: {pred} | {path}")
        try:
            img = Image.open(path).convert("RGB")
            plt.imshow(img)
            plt.title(f"Entropy: {ent:.2f}\nTrue: {true} | Pred: {pred}")
            plt.axis("off")
            plt.savefig(os.path.join(outlier_dir, f"{task}_outlier_{i+1}.png"))
            plt.close()
        except Exception as e:
            print(f"Failed to save image: {path}, error: {e}")

    # === Output class-wise summary ===
    print("\nClass-wise uncertainty and error analysis:")
    print(f"{'Class':<30} {'Errors':<7} {'Uncertain':<10} {'Total':<6} {'Err%':<6} {'Unc%':<6}")
    for cls in class_names:
        total = total_count[cls]
        if total == 0: continue
        err = error_count[cls]
        ent = entropy_count[cls]
        print(f"{cls:<30} {err:<7} {ent:<10} {total:<6} {err/total:.2%} {ent/total:.2%}")


In [ ]:
analyze_model_uncertainty(file="Artist", task="artist", top_n=20)

Available images: 5706 / Total: 5706, Missing: 0


c:\Users\user\anaconda3\envs\cuda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\cuda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Analyzing: 100%|██████████| 5706/5706 [12:21<00:00,  7.69it/s]



Top 20 most uncertain predictions (possible outliers):
[1] Entropy: 2.2149 | True: 14 | Pred: 21 | datasets\wikiart\Realism\nicholas-roerich_study-of-landscape.jpg
[2] Entropy: 2.1996 | True: 13 | Pred: 4 | datasets\wikiart\Expressionism\martiros-saryan_a-bull-1922.jpg
[3] Entropy: 2.1906 | True: 15 | Pred: 20 | datasets\wikiart\Expressionism\pablo-picasso_self-portrait-1901.jpg
[4] Entropy: 2.1607 | True: 16 | Pred: 1 | datasets\wikiart\Post_Impressionism\paul-cezanne_madame-cezanne-in-a-striped-rob.jpg
[5] Entropy: 2.1543 | True: 4 | Pred: 8 | datasets\wikiart\Impressionism\claude-monet_the-dinner-1869.jpg
[6] Entropy: 2.1357 | True: 15 | Pred: 3 | datasets\wikiart\Expressionism\pablo-picasso_madame-soler-1905.jpg
[7] Entropy: 2.1291 | True: 11 | Pred: 7 | datasets\wikiart\Realism\john-singer-sargent_the-daughters-of-edward-darley-boit-1882.jpg
[8] Entropy: 2.0827 | True: 7 | Pred: 22 | datasets\wikiart\Romanticism\gustave-dore_illustration-for-charles-perraults-bluebeard.jpg
[9] En

In [5]:
analyze_model_uncertainty(file="Genre", task="genre", top_n=20)

Available images: 19492 / Total: 19492, Missing: 0


c:\Users\user\anaconda3\envs\cuda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\cuda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Analyzing: 100%|██████████| 19492/19492 [08:20<00:00, 38.98it/s]



Top 20 most uncertain predictions (possible outliers):
[1] Entropy: 2.1726 | True: 3 | Pred: 2 | datasets\wikiart\Symbolism\mikhail-vrubel_head-of-demon-1.jpg
[2] Entropy: 2.1573 | True: 9 | Pred: 2 | datasets\wikiart\Synthetic_Cubism\georges-braque_the-clarinet-valse-1912.jpg
[3] Entropy: 2.1527 | True: 0 | Pred: 2 | datasets\wikiart\Expressionism\ilka-gedo_portrait-of-kl-ri-horv-th-1972.jpg
[4] Entropy: 2.1480 | True: 1 | Pred: 2 | datasets\wikiart\Impressionism\john-singer-sargent_venetian-canal.jpg
[5] Entropy: 2.1473 | True: 2 | Pred: 2 | datasets\wikiart\Expressionism\balthus_girl-at-the-window-1955.jpg
[6] Entropy: 2.1336 | True: 5 | Pred: 7 | datasets\wikiart\Pointillism\henri-edmond-cross_aguttes.jpg
[7] Entropy: 2.1285 | True: 0 | Pred: 4 | datasets\wikiart\Abstract_Expressionism\philip-guston_close-up-iii-1961.jpg
[8] Entropy: 2.1233 | True: 3 | Pred: 3 | datasets\wikiart\Expressionism\m.c.-escher_wateringcan.jpg
[9] Entropy: 2.1232 | True: 7 | Pred: 2 | datasets\wikiart\Na

In [ ]:
analyze_model_uncertainty(task="style", top_n=20)